# Enriquecimento de Clientes com Informações de Estados

Este notebook enriquece a tabela `dim_clientes` com informações adicionais dos estados brasileiros usando a API BrasilAPI.

**Objetivo**: Criar uma view enriquecida com dados de estados (nome completo, região, etc.) para análises mais detalhadas.


## 1. Importar Bibliotecas


In [3]:
import duckdb
import os
import pandas as pd
import requests
import json
from typing import Dict, List


## 2. Conectar ao DuckDB


In [4]:
# Caminho do banco de dados (compartilhado com outros serviços)
db_path = "/app/lakehouse/lakehouse.duckdb"
os.makedirs(os.path.dirname(db_path), exist_ok=True)

# Conectar ao DuckDB
con = duckdb.connect(db_path)
print(f"✓ Conectado ao DuckDB: {db_path}")


✓ Conectado ao DuckDB: /app/lakehouse/lakehouse.duckdb


## 3. Carregar dados da tabela dim_clientes


In [6]:
# Carregar dados da dimensão de clientes
df_clientes = con.execute("SELECT * FROM dim_clientes").fetchdf()

print(f"✓ Carregados {len(df_clientes)} clientes")
print(f"\nEstados únicos encontrados: {df_clientes['cliente_estado'].nunique()}")
print(f"\nEstados: {sorted(df_clientes['cliente_estado'].unique().tolist())}")

# Visualizar primeiras linhas
df_clientes.head()


✓ Carregados 2517 clientes

Estados únicos encontrados: 27

Estados: ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']


,cliente_id,cliente_nome,cliente_email,cliente_cidade,cliente_estado,total_compras,valor_total_gasto,ticket_medio,primeira_compra,ultima_compra,categorias_compradas,canais_utilizados,total_itens_comprados,segmento_cliente,ranking_cliente
0,3636,Luiz Felipe da Costa,jteixeira@example.net,da Costa Alegre,MG,30,367322.10,12244.070000,2023-09-04,2024-06-15,1,1,105.0,VIP,1
1,2168,Vitor Gabriel Souza,luccateixeira@example.com,Gonçalves Grande,TO,45,339103.65,7535.636667,2023-04-26,2024-03-10,2,2,150.0,VIP,2
2,8809,Stella Caldeira,rebeca68@example.net,da Rosa do Amparo,SP,45,320905.05,7131.223333,2023-08-08,2024-08-14,2,3,180.0,VIP,3
3,3676,Stephany Silveira,oliveiramiguel@example.com,das Neves de Campos,MA,60,318472.05,5307.867500,2023-03-03,2024-08-17,3,4,210.0,VIP,4
4,4187,Esther Moura,miguel80@example.net,Lopes,PB,30,316622.55,10554.085000,2023-05-04,2023-11-07,1,2,120.0,VIP,5


## 4. Buscar informações de estados da API BrasilAPI


In [7]:
# Buscar todos os estados da API BrasilAPI
url_estados = "https://brasilapi.com.br/api/ibge/uf/v1"

print(f"Buscando informações de estados da API BrasilAPI...")
print(f"URL: {url_estados}")

try:
    response = requests.get(url_estados, timeout=10)
    response.raise_for_status()
    estados_api = response.json()
    
    print(f"✓ {len(estados_api)} estados encontrados na API")
    
    # Criar DataFrame com informações dos estados
    df_estados = pd.DataFrame(estados_api)
    
    # Visualizar estrutura dos dados
    print(f"\nColunas disponíveis: {df_estados.columns.tolist()}")
    df_estados.head()
    
except Exception as e:
    print(f"❌ Erro ao buscar dados da API: {e}")
    raise


Buscando informações de estados da API BrasilAPI...
URL: https://brasilapi.com.br/api/ibge/uf/v1
✓ 27 estados encontrados na API

Colunas disponíveis: ['id', 'sigla', 'nome', 'regiao']


## 5. Preparar dados para enriquecimento


In [8]:
# Verificar estrutura dos dados de estados
print("Estrutura dos dados de estados:")
print(df_estados.info())
print("\nPrimeiras linhas:")
print(df_estados.head())

# Verificar se temos a sigla (sigla ou id) para fazer o join
print(f"\nColunas disponíveis: {df_estados.columns.tolist()}")

# Normalizar nome da coluna de sigla (pode ser 'sigla' ou 'id')
if 'sigla' in df_estados.columns:
    df_estados = df_estados.rename(columns={'sigla': 'estado_sigla'})
elif 'id' in df_estados.columns:
    df_estados = df_estados.rename(columns={'id': 'estado_sigla'})

# Garantir que temos a sigla em maiúsculas para fazer o join
if 'estado_sigla' in df_estados.columns:
    df_estados['estado_sigla'] = df_estados['estado_sigla'].str.upper()
else:
    # Se não tiver sigla, tentar usar a primeira coluna como identificador
    print("⚠ Aviso: Coluna 'sigla' não encontrada. Usando primeira coluna como identificador.")
    df_estados['estado_sigla'] = df_estados.iloc[:, 0].str.upper()

print(f"\n✓ Dados de estados preparados")
print(f"Estados disponíveis: {sorted(df_estados['estado_sigla'].unique().tolist())}")


Estrutura dos dados de estados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      27 non-null     int64 
 1   sigla   27 non-null     object
 2   nome    27 non-null     object
 3   regiao  27 non-null     object
dtypes: int64(1), object(3)
memory usage: 996.0+ bytes
None

Primeiras linhas:
   id sigla      nome                                    regiao
0  11    RO  Rondônia  {'id': 1, 'sigla': 'N', 'nome': 'Norte'}
1  12    AC      Acre  {'id': 1, 'sigla': 'N', 'nome': 'Norte'}
2  13    AM  Amazonas  {'id': 1, 'sigla': 'N', 'nome': 'Norte'}
3  14    RR   Roraima  {'id': 1, 'sigla': 'N', 'nome': 'Norte'}
4  15    PA      Pará  {'id': 1, 'sigla': 'N', 'nome': 'Norte'}

Colunas disponíveis: ['id', 'sigla', 'nome', 'regiao']

✓ Dados de estados preparados
Estados disponíveis: ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 

## 6. Fazer join e enriquecer dados


In [9]:
# Normalizar sigla do estado nos clientes para maiúsculas
df_clientes['cliente_estado'] = df_clientes['cliente_estado'].str.upper()

# Fazer join com informações dos estados
df_enriquecido = df_clientes.merge(
    df_estados,
    left_on='cliente_estado',
    right_on='estado_sigla',
    how='left'
)

print(f"✓ Join realizado: {len(df_enriquecido)} registros")
print(f"\nColunas adicionadas: {set(df_enriquecido.columns) - set(df_clientes.columns)}")

# Visualizar resultado
df_enriquecido.head()


✓ Join realizado: 2517 registros

Colunas adicionadas: {'regiao', 'estado_sigla', 'id', 'nome'}


,cliente_id,cliente_nome,cliente_email,cliente_cidade,cliente_estado,total_compras,valor_total_gasto,ticket_medio,primeira_compra,ultima_compra,categorias_compradas,canais_utilizados,total_itens_comprados,segmento_cliente,ranking_cliente,id,estado_sigla,nome,regiao
0,3636,Luiz Felipe da Costa,jteixeira@example.net,da Costa Alegre,MG,30,367322.10,12244.070000,2023-09-04,2024-06-15,1,1,105.0,VIP,1,31,MG,Minas Gerais,"{'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}"
1,2168,Vitor Gabriel Souza,luccateixeira@example.com,Gonçalves Grande,TO,45,339103.65,7535.636667,2023-04-26,2024-03-10,2,2,150.0,VIP,2,17,TO,Tocantins,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
2,8809,Stella Caldeira,rebeca68@example.net,da Rosa do Amparo,SP,45,320905.05,7131.223333,2023-08-08,2024-08-14,2,3,180.0,VIP,3,35,SP,São Paulo,"{'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}"
3,3676,Stephany Silveira,oliveiramiguel@example.com,das Neves de Campos,MA,60,318472.05,5307.867500,2023-03-03,2024-08-17,3,4,210.0,VIP,4,21,MA,Maranhão,"{'id': 2, 'sigla': 'NE', 'nome': 'Nordeste'}"
4,4187,Esther Moura,miguel80@example.net,Lopes,PB,30,316622.55,10554.085000,2023-05-04,2023-11-07,1,2,120.0,VIP,5,25,PB,Paraíba,"{'id': 2, 'sigla': 'NE', 'nome': 'Nordeste'}"


## 7. Criar view enriquecida no DuckDB


In [13]:
# Criar tabela temporária com os dados enriquecidos
con.execute("DROP TABLE IF EXISTS dim_clientes_enriquecido_temp")
con.execute("CREATE TABLE dim_clientes_enriquecido_temp AS SELECT * FROM df_enriquecido")

print("✓ Tabela temporária criada")

# Verificar estrutura
result = con.execute("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_name = 'dim_clientes_enriquecido_temp'
    ORDER BY ordinal_position
""").fetchdf()

print("\nEstrutura da tabela:")
print(result)

result = con.execute("""SELECT * FROM dim_clientes_enriquecido_temp LIMIT 10""").fetchdf()
display(result)

✓ Tabela temporária criada

Estrutura da tabela:
              column_name                                        data_type
0              cliente_id                                          INTEGER
1            cliente_nome                                          VARCHAR
2           cliente_email                                          VARCHAR
3          cliente_cidade                                          VARCHAR
4          cliente_estado                                          VARCHAR
5           total_compras                                           BIGINT
6       valor_total_gasto                                           DOUBLE
7            ticket_medio                                           DOUBLE
8         primeira_compra                                        TIMESTAMP
9           ultima_compra                                        TIMESTAMP
10   categorias_compradas                                           BIGINT
11      canais_utilizados                          

,cliente_id,cliente_nome,cliente_email,cliente_cidade,cliente_estado,total_compras,valor_total_gasto,ticket_medio,primeira_compra,ultima_compra,categorias_compradas,canais_utilizados,total_itens_comprados,segmento_cliente,ranking_cliente,id,estado_sigla,nome,regiao
0,3636,Luiz Felipe da Costa,jteixeira@example.net,da Costa Alegre,MG,30,367322.10,12244.070000,2023-09-04,2024-06-15,1,1,105.0,VIP,1,31,MG,Minas Gerais,"{'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}"
1,2168,Vitor Gabriel Souza,luccateixeira@example.com,Gonçalves Grande,TO,45,339103.65,7535.636667,2023-04-26,2024-03-10,2,2,150.0,VIP,2,17,TO,Tocantins,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
2,8809,Stella Caldeira,rebeca68@example.net,da Rosa do Amparo,SP,45,320905.05,7131.223333,2023-08-08,2024-08-14,2,3,180.0,VIP,3,35,SP,São Paulo,"{'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}"
3,3676,Stephany Silveira,oliveiramiguel@example.com,das Neves de Campos,MA,60,318472.05,5307.867500,2023-03-03,2024-08-17,3,4,210.0,VIP,4,21,MA,Maranhão,"{'id': 2, 'sigla': 'NE', 'nome': 'Nordeste'}"
4,4187,Esther Moura,miguel80@example.net,Lopes,PB,30,316622.55,10554.085000,2023-05-04,2023-11-07,1,2,120.0,VIP,5,25,PB,Paraíba,"{'id': 2, 'sigla': 'NE', 'nome': 'Nordeste'}"
5,9827,Vitor Hugo Pires,qnovaes@example.org,da Cruz de Teixeira,RR,30,309573.30,10319.110000,2023-02-26,2024-05-15,1,2,90.0,VIP,6,14,RR,Roraima,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
6,4643,Leonardo Silveira,melissa64@example.org,Sales,PB,30,293875.20,9795.840000,2023-01-24,2024-04-02,1,2,90.0,VIP,7,25,PB,Paraíba,"{'id': 2, 'sigla': 'NE', 'nome': 'Nordeste'}"
7,2353,Srta. Larissa Pinto,rpereira@example.org,Rocha,RS,45,290109.00,6446.866667,2023-02-24,2024-09-06,3,2,165.0,VIP,8,43,RS,Rio Grande do Sul,"{'id': 4, 'sigla': 'S', 'nome': 'Sul'}"
8,4852,Maria Eduarda Rocha,maria-vitoria45@example.net,Lima,DF,30,286934.10,9564.470000,2023-03-02,2024-01-06,2,2,150.0,VIP,9,53,DF,Distrito Federal,"{'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}"
9,6355,Júlia Pereira,caue79@example.net,Rodrigues do Sul,PI,15,286839.00,19122.600000,2024-04-03,2024-04-03,1,1,75.0,VIP,10,22,PI,Piauí,"{'id': 2, 'sigla': 'NE', 'nome': 'Nordeste'}"


In [15]:
# Primeiro, criar uma tabela de estados no DuckDB para fazer o join
# Carregar dados de estados no DuckDB
con.execute("DROP TABLE IF EXISTS estados_brasilapi")
con.execute("CREATE TABLE estados_brasilapi AS SELECT * FROM df_estados")
print("✓ Tabela 'estados_brasilapi' criada")

# Criar view enriquecida (para uso no lab)
# Esta view combina dim_clientes com informações de estados da API
view_sql = """
CREATE OR REPLACE VIEW dim_clientes_enriquecido AS
SELECT 
    c.*,
    -- Informações do estado da API BrasilAPI
    e.nome as estado_nome_completo,
    e.regiao as estado_regiao,
    e.regiao_sigla as estado_regiao_sigla
FROM dim_clientes c
LEFT JOIN estados_brasilapi e
    ON UPPER(c.cliente_estado) = UPPER(e.estado_sigla)
"""

con.execute(view_sql)
print("✓ View 'dim_clientes_enriquecido' criada com sucesso!")


✓ Tabela 'estados_brasilapi' criada


BinderException: Binder Error: Table "e" does not have a column named "regiao_sigla"
LINE 8:     e.regiao_sigla as estado_regiao_sigla
            ^

## 8. Verificar a view criada


In [ ]:
# Consultar a view criada
df_view = con.execute("SELECT * FROM dim_clientes_enriquecido LIMIT 10").fetchdf()

print("Primeiras 10 linhas da view enriquecida:")
print(df_view.to_string())

print(f"\n✓ View criada com {len(con.execute('SELECT COUNT(*) FROM dim_clientes_enriquecido').fetchone()[0])} registros")


## 9. Análise rápida por região


In [5]:
# Análise de clientes por região
analise_regiao = con.execute("""
    SELECT 
        estado_regiao,
        COUNT(DISTINCT cliente_id) as total_clientes,
        SUM(valor_total_gasto) as receita_total,
        AVG(valor_total_gasto) as ticket_medio,
        AVG(total_compras) as compras_media_por_cliente
    FROM dim_clientes_enriquecido
    WHERE estado_regiao IS NOT NULL
    GROUP BY estado_regiao
    ORDER BY receita_total DESC
""").fetchdf()

print("Análise de clientes por região:")
print(analise_regiao.to_string(index=False))
con.close()

CatalogException: Catalog Error: Table with name dim_clientes_enriquecido does not exist!
Did you mean "dim_clientes"?
LINE 8:     FROM dim_clientes_enriquecido
                 ^

## 10. Resumo

A view `dim_clientes_enriquecido` está pronta para uso no lab! Ela contém:
- Todas as colunas originais de `dim_clientes`
- Informações adicionais dos estados:
  - `estado_nome_completo`: Nome completo do estado
  - `estado_regiao`: Nome da região (Norte, Nordeste, etc.)
  - `estado_regiao_sigla`: Sigla da região (N, NE, etc.)

**Nota**: Esta view é criada apenas no ambiente do lab e não afeta as tabelas do dbt.
